<a href="https://colab.research.google.com/github/LukeWeidenwalker/fastai-notebooks/blob/master/08_movielens_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastai

     |████████████████████████████████| 204kB 8.3MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 


In [2]:
from fastai.vision.all import *

In [3]:
path = untar_data(URLs.ML_100k)

In [4]:
path.ls()

(#23) [Path('/root/.fastai/data/ml-100k/u.genre'),Path('/root/.fastai/data/ml-100k/u.occupation'),Path('/root/.fastai/data/ml-100k/u.info'),Path('/root/.fastai/data/ml-100k/u.user'),Path('/root/.fastai/data/ml-100k/mku.sh'),Path('/root/.fastai/data/ml-100k/ub.test'),Path('/root/.fastai/data/ml-100k/u3.test'),Path('/root/.fastai/data/ml-100k/ub.base'),Path('/root/.fastai/data/ml-100k/u1.base'),Path('/root/.fastai/data/ml-100k/u4.base')...]

Path('/root/.fastai/data/ml-100k/README')

In [7]:
with open(path / "README") as readme:
  for row in readme:
    print(row)

SUMMARY & USAGE LICENSE




MovieLens data sets were collected by the GroupLens Research Project

at the University of Minnesota.

 

This data set consists of:

	* 100,000 ratings (1-5) from 943 users on 1682 movies. 

	* Each user has rated at least 20 movies. 

        * Simple demographic info for the users (age, gender, occupation, zip)



The data was collected through the MovieLens web site

(movielens.umn.edu) during the seven-month period from September 19th, 

1997 through April 22nd, 1998. This data has been cleaned up - users

who had less than 20 ratings or did not have complete demographic

information were removed from this data set. Detailed descriptions of

the data file can be found at the end of this file.



Neither the University of Minnesota nor any of the researchers

involved can guarantee the correctness of the data, its suitability

for any particular purpose, or the validity of results based on the

use of the data set.  The data set may be used for any resea